In [1]:
import pandas as pd
import win32com.client
import os

In [2]:
#もし発言録まとめの列名が変更された時のために
KOUTEI_ID = "koutei_ID"
SAGYO_ID = "sagyo_ID"
KOUDOU_ID = "koudou_ID"
#列を指定する際にループで増加させるインデックスの増加定数
#
INDEX_PLUS = 5

In [3]:
file_name = "中間報告用発言録まとめ.xlsx"
sheet_names = pd.ExcelFile(file_name).sheet_names
excels = pd.read_excel(file_name, sheet_name=sheet_names)

In [4]:
xlApp = win32com.client.Dispatch("Excel.Application")

In [5]:
colmuns = [KOUTEI_ID, SAGYO_ID, KOUDOU_ID]
IDs = pd.DataFrame(index=[], columns=colmuns)
for sheet_name in sheet_names:
    excel = excels[sheet_name]
    excel[KOUTEI_ID] = excel[KOUTEI_ID].fillna(method="ffill").astype(int)
    excel[SAGYO_ID] = excel[SAGYO_ID].fillna(method="ffill").astype(int)
    excel[KOUDOU_ID] = excel[KOUDOU_ID].fillna(method="ffill").astype(int)
    sub_IDs = excel[colmuns]
    IDs = IDs.append(sub_IDs)
IDs = IDs.drop_duplicates().sort_values([KOUTEI_ID,SAGYO_ID,KOUDOU_ID]).reset_index(drop=True)
IDs

,koutei_ID,sagyo_ID,koudou_ID
0,1,1,1
1,1,1,2
2,1,2,1
3,1,3,1
4,1,3,2
5,1,4,1
6,1,5,1
7,1,6,1
8,2,1,1
9,2,1,2


In [18]:
wb = xlApp.Workbooks.Add()
ws = wb.sheets[0]
ws.Name = "比較技能分析表"
xlApp.Visible = True

In [19]:
#header作成
header = [
    "工程",
    "作業",
    "具体的な行動\n共通人数",
    "行動のポイント\n共通人数",
    "行動の背景\n共通人数",
    "比較"
]
ws.Range(ws.Cells(2,2),ws.Cells(2,7)).Value = header
ws.Range(ws.Cells(2,2),ws.Cells(2,7)).HorizontalAlignment = 3
for i, sheet_name in enumerate(sheet_names):
    i += 1 #excelの列インデックスに使うため
    ws.Cells(2,i*3+INDEX_PLUS).Value = sheet_name
    target_range = ws.Range(ws.Cells(2,i*3+INDEX_PLUS),ws.Cells(2,i*3+INDEX_PLUS+2))
    target_range.MergeCells = True
    target_range.HorizontalAlignment = 3
    target_range.ColumnWidth = 30

In [20]:
#技能分析表作成
koutei_ID = 0
sagyo_ID = 0
koutei_st = 3
sagyo_st = 3
img_size = 5#画像のheight(cm)
for row in range(1):#range(len(IDs)):
    for sub_cnt, sheet_name in enumerate(sheet_names):
        sub_cnt += 1 #excelの列インデックスに使うため
        excel = excels[sheet_name]
        #koutei_ID,sagyou_ID,Koudou_IDが全て一致するものを抽出
        skill_table = excel[
            (excel[KOUTEI_ID] == IDs[KOUTEI_ID][row]) &
            (excel[SAGYO_ID] == IDs[SAGYO_ID][row]) &
            (excel[KOUDOU_ID] == IDs[KOUDOU_ID][row])
        ]
        #koutei_IDが変わった時に工程列が空白であれば、工程をB列に記入
        if koutei_ID != IDs[KOUTEI_ID][row] & (ws.Cells(koutei_st,2).Value is None):
            ws.Cells(koutei_st,2).Value = skill_table["工程"][0]
            
        #動画があれば、リンクを挿入
        df_video = skill_table.filter(like = "video", axis = 1).reset_index(drop=True)
        if(not(df_video.empty)):
            df_video = df_video.dropna(how="all", axis=1)
            for video_cnt,video_index in enumerate(df_video):
                video_path = os.getcwd() + "/videos/" + df_video[video_index][0]
                #ディレクトリにファイルが存在する場合はリンクを挿入
                if os.path.exists(video_path):
                    ws.Cells(sagyo_st, sub_cnt*3+5+video_cnt).Value = df_video[video_index][0]
                    ws.Hyperlinks.Add(
                        Anchor = ws.Cells(sagyo_st, sub_cnt*3+INDEX_PLUS+video_cnt), 
                        Address = video_path
                    )
                else:
                    print(video_path, "なんてないで！！")
                    
        #画像があれば挿入
        df_image = skill_table.filter(like = "image", axis = 1)
        if(not(df_image.empty)):
            df_image = df_image.dropna(how="all", axis=1).reset_index(drop=True)
            image_row = sagyo_st + 1#画像を挿入する行数
            #画像行の行高さを調整 img_size*33
            ws.Rows(image_row).RowHeight = img_size * 33 
            for image_cnt,image_index in enumerate(df_image):
                image_path = os.getcwd() + "/images/" + df_image[image_index][0]
                #画像3枚ごとに次の行に挿入
                if image_cnt / 3 == 1:
                    image_row += 1
                    #画像行の行高さを調整 img_size*33
                    ws.Rows(image_row).RowHeight = img_size * 33
                #ディレクトリにファイルが存在する場合は画像を挿入
                if os.path.exists(image_path):
                    image = ws.Shapes.AddPicture(
                        Filename = os.getcwd() + "/images/" + df_image[image_index][0],
                        LinkToFile = False,
                        SaveWithDocument = True,
                        Left = ws.Cells(image_row, sub_cnt*3+INDEX_PLUS+image_cnt).Left,
                        Top = ws.Cells(image_row, sub_cnt*3+INDEX_PLUS+image_cnt).Top,
                        Width = -1,
                        Height= -1
                    )
                    #画像番号入力
                    ws.Cells(image_row, sub_cnt*3+INDEX_PLUS+image_cnt).value = image_cnt + 1
                    ws.Cells(image_row, sub_cnt*3+INDEX_PLUS+image_cnt).HorizontalAlignment = 2
                    ws.Cells(image_row, sub_cnt*3+INDEX_PLUS+image_cnt).VerticalAlignment = 1
                    
                    #画像サイズ調整
                    image.Height = 28.34646 * img_size
                    if image.Width > 28.34646 * 6:#img_sizeで指定した幅(cm)のアスペクト比計算で幅が6cmを超える場合は7cmに設定
                        image.Width = 28.34646 * 6
                    
                    #画像をセルの中央へ配置
                    image.Left = ws.Cells(image_row,sub_cnt*3+INDEX_PLUS+image_cnt).Left \
                        + (ws.Cells(image_row,sub_cnt*3+INDEX_PLUS+image_cnt).Width - image.Width) / 2
                    image.Top = ws.Cells(image_row,sub_cnt*3+INDEX_PLUS+image_cnt).Top\
                        + (ws.Cells(image_row,sub_cnt*3+INDEX_PLUS+image_cnt).Height - image.Height) / 2
                else:
                    print(image_path, "なんてないで！！")
           
    #miniheader記入
    minihead_row = ws.UsedRange.Rows.Count + 2
    miniheader = ["具体的な行動", "行動のポイント", "行動の背景"]
    for i in range(len(sheet_names)):
        i += 1 #excelの列インデックスに使うため
        target_range = ws.Range(ws.Cells(minihead_row, i*3+INDEX_PLUS), ws.Cells(minihead_row, i*3+INDEX_PLUS+2))
        target_range.Value = miniheader
        target_range.HorizontalAlignment = 3
        
    #技能分析表の内容を記入
    skill_table_row = minihead_row + 1
    #一人ずつ内容を埋めていく
    for sub_cnt, sheet_name in enumerate(sheet_names):
        sub_cnt += 1 #excelの列インデックスに使うため
        excel = excels[sheet_name]
        #koutei_ID,sagyou_ID,Koudou_IDが全て一致するものを抽出
        skill_table = excel[
            (excel[KOUTEI_ID] == IDs[KOUTEI_ID][row]) &
            (excel[SAGYO_ID] == IDs[SAGYO_ID][row]) &
            (excel[KOUDOU_ID] == IDs[KOUDOU_ID][row])
        ]
        #内容記載に必要部分のみ抽出
        skill_table = skill_table.loc[:, ["具体的な行動の仕方", "行動のポイント", "ポイントの背景"]]
        #行動の行数
        num_koudou = skill_table.shape[0]
        #内容を埋めていく
        if num_koudou == 1:
            ws.Range(ws.Cells(skill_table_row, sub_cnt*3+INDEX_PLUS),
                     ws.Cells(skill_table_row, sub_cnt*3+INDEX_PLUS+2)).value = skill_table
        else:
            ws.Range(ws.Cells(skill_table_row, sub_cnt*3+INDEX_PLUS),
                     ws.Cells(skill_table_row+num_koudou, sub_cnt*3+INDEX_PLUS+2)).value = skill_table

    sagyo_st = ws.UsedRange.Rows.Count + 2

TypeError: Objects for SAFEARRAYS must be sequences (of sequences), or a buffer object.

In [33]:
len(df_image)
print(ws.UsedRange.Rows.Count)

3


In [16]:
ws.UsedRange.Rows.Count

1

In [76]:
skill_table = excel[
            (excel[KOUTEI_ID] == IDs[KOUTEI_ID][0]) &
            (excel[SAGYO_ID] == IDs[SAGYO_ID][0]) &
            (excel[KOUDOU_ID] == IDs[KOUDOU_ID][0])
        ]
df_image = skill_table.filter(like = "image", axis = 1)
df_image = df_image.dropna(how="all", axis=1).reset_index(drop=True)
skill_table = skill_table.loc[:,["具体的な行動の仕方","行動のポイント","ポイントの背景"]]
ws.Range("D3:F3").value = skill_table.iloc[0,:]

In [12]:
xlApp.Quit()